In [1]:
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv


In [21]:
%%file flight_preprocess.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class MRProcess(MRJob):
    def mapper(self, _, line):

        (year, month, day, day_of_week, airline, flight_number, tail_number,
        origin_airport, destination_airport, scheduled_departure, departure_time,
        departure_delay, taxi_out, wheels_off, scheduled_time, elapsed_time, air_time,
        distance, wheels_on, taxi_in, scheduled_arrival, arrival_time, arrival_delay,
        diverted, cancelled, cancellation_reason, air_system_delay, security_delay,
        airline_delay, late_aircraft_delay, weather_delay) = line.split(',')
        yield "items", (year, month, day)
if __name__ == "__main__":
    MRProcess.run()


Overwriting flight_preprocess.py


In [29]:
%%file flight_preprocess.py

from mrjob.job import MRJob
import csv

class MRAverageDistance(MRJob):

    def mapper(self, _, line):
        # Use CSV reader to correctly handle commas inside quotations
        reader = csv.reader([line])
        for row in reader:
            try:
                distance = float(row[17]) 
                yield "average_distance", distance
            except ValueError:
                # Skip the header or lines with missing data
                continue

    def reducer(self, key, values):
        total_distance = 0
        count = 0
        for value in values:
            total_distance += value
            count += 1
        if count > 0:
            average_distance = total_distance / count
            yield key, average_distance

if __name__ == '__main__':
    MRAverageDistance.run()


Overwriting flight_preprocess.py


In [31]:
%%file flight_preprocess.py
from mrjob.job import MRJob
import csv

class MRAverageDistance(MRJob):

    def mapper(self, _, line):
        reader = csv.reader([line])
        for row in reader:
            if row[17].isdigit():
                distance = float(row[17]) 
                yield "average_distance", distance

    def reducer(self, key, values):
        total_distance = 0
        count = 0
        for value in values:
            total_distance += value
            count += 1
        if count > 0:
            average_distance = total_distance / count
            yield key, average_distance

if __name__ == '__main__':
    MRAverageDistance.run()


Overwriting flight_preprocess.py


In [33]:
%%file flight_distance_average.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class MRAverageDistance(MRJob):

    def mapper(self, _, line):
        # Using tuple unpacking to extract each field from the line
        (year, month, day, day_of_week, airline, flight_number, tail_number,
         origin_airport, destination_airport, scheduled_departure, departure_time,
         departure_delay, taxi_out, wheels_off, scheduled_time, elapsed_time, air_time,
         distance, wheels_on, taxi_in, scheduled_arrival, arrival_time, arrival_delay,
         diverted, cancelled, cancellation_reason, air_system_delay, security_delay,
         airline_delay, late_aircraft_delay, weather_delay) = line.split(',')

        # Convert distance to float and yield
        try:
            distance = float(distance)
            yield "average_distance", distance
        except ValueError:
            # Skip lines where distance cannot be converted to float (unexpectedly malformed data)
            pass

    def reducer(self, key, values):
        total_distance = 0
        count = 0
        for value in values:
            total_distance += value
            count += 1
        if count > 0:
            average_distance = total_distance / count
            yield key, average_distance

if __name__ == '__main__':
    MRAverageDistance.run()


Writing flight_distance_average.py


In [37]:
%%file flight_airline_distance_average.py

from mrjob.job import MRJob
import csv

class MRAirlineDistanceAverage(MRJob):

    def mapper(self, _, line):
        # Using tuple unpacking to extract each field from the line
        (year, month, day, day_of_week, airline, flight_number, tail_number,
         origin_airport, destination_airport, scheduled_departure, departure_time,
         departure_delay, taxi_out, wheels_off, scheduled_time, elapsed_time, air_time,
         distance, wheels_on, taxi_in, scheduled_arrival, arrival_time, arrival_delay,
         diverted, cancelled, cancellation_reason, air_system_delay, security_delay,
         airline_delay, late_aircraft_delay, weather_delay) = line.split(',')

        # Convert distance to float and yield with airline as key
        try:
            distance = float(distance)
            yield airline, distance
        except ValueError:
            # Skip lines where distance cannot be converted to float
            pass

    def reducer(self, key, values):
        total_distance = 0
        count = 0
        for value in values:
            total_distance += value
            count += 1
        if count > 0:
            average_distance = total_distance / count
            yield key, average_distance

if __name__ == '__main__':
    MRAirlineDistanceAverage.run()


Overwriting flight_airline_distance_average.py


In [39]:
!python flight_airline_distance_average.py flights.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/flight_airline_distance_average.Karez.20240423.084837.965803
Running step 1 of 1...
job output is in /var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/flight_airline_distance_average.Karez.20240423.084837.965803/output
Streaming final output from /var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/flight_airline_distance_average.Karez.20240423.084837.965803/output...
"NK"	985.2671176275143
"OO"	496.7721639899856
"DL"	853.6218253392869
"EV"	462.25173914335716
"F9"	967.2148597472368
"HA"	632.5918423536816
"MQ"	422.31540701621003
"AS"	1197.4180824363411
"B6"	1062.1751782451095
"UA"	1271.5456844081027
"US"	911.5038623153763
"VX"	1405.9893220037802
"WN"	740.7113345035682
"AA"	1041.3392223520077
Removing temp directory /var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/flight_airline_distance_average.Karez.20240423.084837.965803...


In [ ]:
%%file flight_preprocess.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class MRProcess(MRJob):
    def mapper(self, _, line):

        (year, month, day, day_of_week, airline, flight_number, tail_number,
        origin_airport, destination_airport, scheduled_departure, departure_time,
        departure_delay, taxi_out, wheels_off, scheduled_time, elapsed_time, air_time,
        distance, wheels_on, taxi_in, scheduled_arrival, arrival_time, arrival_delay,
        diverted, cancelled, cancellation_reason, air_system_delay, security_delay,
        airline_delay, late_aircraft_delay, weather_delay) = line.split(',')
        yield "items", (year, month, day)
if __name__ == "__main__":
    MRProcess.run()


In [3]:
distance, animal = '10', 'cat'

In [4]:
distance

'10'

In [6]:
line = "10, cat"

In [12]:
distance1 , animal1 = line.split(',')

In [11]:
distance1

'10'